In [8]:
from casacore.tables import table
import numpy as np

from casatasks import applycal

Summary: TIME is in JD or something. ANTENNA2 is the reference antenna. SCAN_NUMBER is always 1. FLAG needs to be rewritten.

In [49]:
t = table('/lustre/yuping/2020-12-28-dsa2000-take2/test-gain/00.gcal')

Successful readonly open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/test-gain/00.gcal: 13 columns, 8012 rows


In [4]:
t.colnames()

['TIME',
 'FIELD_ID',
 'SPECTRAL_WINDOW_ID',
 'ANTENNA1',
 'ANTENNA2',
 'INTERVAL',
 'SCAN_NUMBER',
 'OBSERVATION_ID',
 'CPARAM',
 'PARAMERR',
 'FLAG',
 'SNR',
 'WEIGHT']

In [31]:
t.getcol('FLAG').shape

(8012, 130, 2)

In [45]:
t.putcol('FLAG', np.full_like(t.getcol('FLAG'), False))

In [35]:
t.putcol('CPARAM', np.full_like(t.getcol('CPARAM'), 2+0j))

In [51]:
t.getcol('CPARAM').shape

(8012, 130, 2)

In [52]:
t.close()

Now let's see what it does to a MS...I tested it out.

So applycal divides the data by the appropriate CPARAM products. If the FLAG column is TRUE, then it doesn't do anything with that antenna-time-channel solution.

In [2]:
from casatasks import applycal

In [47]:
%time applycal(vis='/lustre/yuping/2020-12-28-dsa2000-take2/test-gain/00.ms', gaintable='/lustre/yuping/2020-12-28-dsa2000-take2/test-gain/00.gcal', flagbackup=False, applymode='calonly')

CPU times: user 2min 29s, sys: 2min 21s, total: 4min 51s
Wall time: 10min 18s


In [69]:
phase = np.random.normal(scale=0.1, size=11)
gain = np.exp(-1j * (2 * np.pi) * phase/360)
gain_repeat = np.repeat(gain[:, np.newaxis], 3 * 2, axis=1).reshape(11, 3, 2)
gain_repeat.shape

(11, 3, 2)

In [64]:
gain[:np.newaxis,np.newaxis].shape

(1, 1, 11)

In [8]:
def gauss_phase_per_ant_time(bcal, rms_deg):
    with table(bcal, readonly=False) as t:
        t.putcol('FLAG', np.full_like(t.getcol('FLAG'), False))
        shape = t.getcol('CPARAM').shape
        phase = np.random.normal(scale=rms_deg, size=shape[0])
        gain = np.exp(-1j * (2 * np.pi) * phase/360)
        gain_repeat = np.repeat(gain[:,np.newaxis,np.newaxis], shape[1] * shape[2]).reshape(shape)
        t.putcol('CPARAM', gain_repeat)

In [3]:
import glob

In [7]:
l = glob.glob('/lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/*gcal')

In [8]:
l

['/lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/06.ms.gcal',
 '/lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/10.ms.gcal',
 '/lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/12.ms.gcal',
 '/lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/04.ms.gcal',
 '/lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/09.ms.gcal',
 '/lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/02.ms.gcal',
 '/lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/13.ms.gcal',
 '/lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/00.ms.gcal',
 '/lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/14.ms.gcal',
 '/lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/07.ms.gcal',
 '/lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/01.ms.gcal',
 '/lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/05.ms.gcal',
 '/lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/08.ms.gcal',
 '/lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/03.ms.gcal',
 '/lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/11.ms.gcal']

In [9]:
for c in l:
    gauss_phase_per_ant_time(c, 1)

Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/06.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/10.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/12.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/04.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/09.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/02.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/1-deg-gain/13.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked tab

In [83]:
tt = table('/lustre/yuping/2020-12-28-dsa2000-take2/0-1-deg-gain/07.ms.gcal')

Successful readonly open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/0-1-deg-gain/07.ms.gcal: 13 columns, 8012 rows


In [84]:
np.unique(tt.getcol('FLAG'))

array([False])

In [86]:
np.abs(tt.getcol('CPARAM'))

array([[[1., 1.],
        [1., 1.],
        [1., 1.],
        ...,
        [1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.],
        ...,
        [1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.],
        ...,
        [1., 1.],
        [1., 1.],
        [1., 1.]],

       ...,

       [[1., 1.],
        [1., 1.],
        [1., 1.],
        ...,
        [1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.],
        ...,
        [1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.],
        ...,
        [1., 1.],
        [1., 1.],
        [1., 1.]]], dtype=float32)

In [87]:
tt.close()

In [4]:
l = glob.glob('/lustre/yuping/2020-12-28-dsa2000-take2/5-deg-gain/*gcal')
for c in l:
    gauss_phase_per_ant_time(c, 5)

Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/5-deg-gain/06.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/5-deg-gain/10.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/5-deg-gain/12.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/5-deg-gain/04.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/5-deg-gain/09.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/5-deg-gain/02.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/5-deg-gain/13.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked tab

In [5]:
l = glob.glob('/lustre/yuping/2020-12-28-dsa2000-take2/10-deg-gain/*gcal')
for c in l:
    gauss_phase_per_ant_time(c, 10)

Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/10-deg-gain/06.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/10-deg-gain/10.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/10-deg-gain/12.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/10-deg-gain/04.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/10-deg-gain/09.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/10-deg-gain/02.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-locked table /lustre/yuping/2020-12-28-dsa2000-take2/10-deg-gain/13.ms.gcal: 13 columns, 8012 rows
Successful read/write open of default-loc

# 20210601 make test data for calibration

I FT'd an image first into the model column.

```
wsclean -predict -weight natural -nwlayers 1 -name SKAMid-1000h 20210226M-1350MHz-1chan-1int.ms
```

In [1]:
from casacore import tables
import numpy as np
from casacore.tables import table

In [5]:
from casacore import tables
def copy_and_cleanup(ms, source_col='MODEL_DATA'):
    with table(ms, readonly=False) as t:
        d = t.getcol(source_col)
        t.putcol('DATA', d)
    tables.removeImagingColumns(ms)

In [6]:
# Copy MODEL into DATA
copy_and_cleanup('/safepool/data/test-calibration/20210226M-1350MHz-1chan-1int.ms')

Successful read/write open of default-locked table /safepool/data/test-calibration/20210226M-1350MHz-1chan-1int.ms: 23 columns, 2096128 rows


In [14]:
gauss_phase_per_ant_time('/safepool/data/test-calibration/20210226M-1chan-1int-10deg.bcal', 10.)

Successful read/write open of default-locked table /safepool/data/test-calibration/20210226M-1chan-1int-10deg.bcal: 13 columns, 2048 rows


In [15]:
# DATA -> CORRECTED_DATA now corrupted
applycal(vis='/safepool/data/test-calibration/20210226M-1350MHz-1chan-1int-10deg.ms',
         gaintable='/safepool/data/test-calibration/20210226M-1chan-1int-10deg.bcal', flagbackup=False, applymode='calonly')

Then copy the corrected_data column to data, and remove extra columns.

In [16]:
copy_and_cleanup('/safepool/data/test-calibration/20210226M-1350MHz-1chan-1int-10deg.ms', source_col='CORRECTED_DATA')

Successful read/write open of default-locked table /safepool/data/test-calibration/20210226M-1350MHz-1chan-1int-10deg.ms: 23 columns, 2096128 rows


In [6]:
def add_noise(ms, rms):
    with table(ms, readonly=False) as t:
        d = t.getcol('DATA')
        print(d.shape)
        noise = np.random.normal(scale=rms, size=d.shape + (2,)).view(np.complex128).astype(np.complex64).reshape(d.shape)
        print(noise.shape)
        d = d + noise
        t.putcol("DATA", d)

The image plane noise for a single channel single timestep should be

In [24]:
2e-6*np.sqrt(8000)*np.sqrt(900/1.5)

0.004381780460041329

In [21]:
add_noise('/safepool/data/test-calibration/20210226M-1350MHz-1chan-1int-1deg.ms', 8.9446)

Successful read/write open of default-locked table /safepool/data/test-calibration/20210226M-1350MHz-1chan-1int-1deg.ms: 22 columns, 2096128 rows
(2096128, 1, 4)
(2096128, 1, 4)


In [25]:
add_noise('/safepool/data/test-calibration/20210226M-1350MHz-1chan-1int-10deg.ms', 8.9446)

Successful read/write open of default-locked table /safepool/data/test-calibration/20210226M-1350MHz-1chan-1int-10deg.ms: 22 columns, 2096128 rows
(2096128, 1, 4)
(2096128, 1, 4)


# For the VLASS sim

Make the noise so that I have a copy

In [7]:
# Copy MODEL into DATA
copy_and_cleanup('/fastpool/yuping/vlass_full.ms')

Successful read/write open of default-locked table /fastpool/yuping/vlass_full.ms: 23 columns, 46683 rows


In [8]:
add_noise('/fastpool/yuping/vlass_full.ms', 0.6678)

Successful read/write open of default-locked table /fastpool/yuping/vlass_full.ms: 22 columns, 46683 rows
(46683, 1000, 4)
(46683, 1000, 4)


# A cotrolled corruption
because I'll need to compare them to the ground truth so that we can see the effects of the artifacts.

In [4]:
with table('/safepool/data/test-calibration/noise.ms', readonly=False) as t:
    d = t.getcol('DATA')
    print(d.shape)
    noise = np.random.normal(scale=8.9446, size=d.shape + (2,)).view(np.complex128).astype(np.complex64).reshape(d.shape)
    t.putcol("DATA", noise)

Successful read/write open of default-locked table /safepool/data/test-calibration/noise.ms: 22 columns, 2096128 rows
(2096128, 1, 4)


Order of operation is corruption first and noise second.

In [6]:
def apply_cal_add_noise(ms, bcal):
    applycal(vis=ms,
         gaintable=bcal, flagbackup=False, applymode='calonly')
    copy_and_cleanup(ms, source_col='CORRECTED_DATA')
    with table('/safepool/data/test-calibration/noise.ms') as t:
        noise = t.getcol('DATA')
    with table(ms, readonly=False) as t:
        d = t.getcol('DATA')
        d = d + noise
        t.putcol("DATA", d)

In [9]:
apply_cal_add_noise('/safepool/data/test-calibration/20210226M-1350MHz-1chan-1int-1deg.ms',
                   '/safepool/data/test-calibration/answers/20210226M-1chan-1int-1deg.bcal')

Successful read/write open of default-locked table /safepool/data/test-calibration/20210226M-1350MHz-1chan-1int-1deg.ms: 23 columns, 2096128 rows
Successful readonly open of default-locked table /safepool/data/test-calibration/noise.ms: 22 columns, 2096128 rows
Successful read/write open of default-locked table /safepool/data/test-calibration/20210226M-1350MHz-1chan-1int-1deg.ms: 22 columns, 2096128 rows


In [10]:
apply_cal_add_noise('/safepool/data/test-calibration/20210226M-1350MHz-1chan-1int-10deg.ms',
                   '/safepool/data/test-calibration/answers/20210226M-1chan-1int-10deg.bcal')

Successful read/write open of default-locked table /safepool/data/test-calibration/20210226M-1350MHz-1chan-1int-10deg.ms: 23 columns, 2096128 rows
Successful readonly open of default-locked table /safepool/data/test-calibration/noise.ms: 22 columns, 2096128 rows
Successful read/write open of default-locked table /safepool/data/test-calibration/20210226M-1350MHz-1chan-1int-10deg.ms: 22 columns, 2096128 rows


In [16]:
def subtract_noise_and_truth(ms):
    with table('/safepool/data/test-calibration/noise.ms') as t:
        noise = t.getcol('DATA')

    with table('/safepool/data/test-calibration/20210226M-1350MHz-1chan-1int-ground-truth.ms') as t:
        truth = t.getcol('DATA')

    with table(ms, readonly=False) as t:
        d = t.getcol('DATA')
        d = d - noise - truth
        t.putcol("DATA", d)

In [17]:
subtract_noise_and_truth('/safepool/data/test-calibration/20210226M-1350MHz-1chan-1int-1deg-artifacts-only.ms')

Successful readonly open of default-locked table /safepool/data/test-calibration/noise.ms: 22 columns, 2096128 rows
Successful readonly open of default-locked table /safepool/data/test-calibration/20210226M-1350MHz-1chan-1int-ground-truth.ms: 22 columns, 2096128 rows
Successful read/write open of default-locked table /safepool/data/test-calibration/20210226M-1350MHz-1chan-1int-1deg-artifacts-only.ms: 22 columns, 2096128 rows
